In [91]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [92]:
tf.__version__

'2.5.0'

In [93]:
tf.test.is_gpu_available()

True

# Data Pre-Processing 

In [94]:
# sets up the data generator with the required transformations
train_datagen = ImageDataGenerator(
    rescale=1./255,     # deafults from doc
    shear_range=0.2,    # deafults from doc
    zoom_range=0.2,      # deafults from doc
    horizontal_flip=True)

# Imports the training data set from the directories and resizes the images (for faster performance)
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',     # path to training set
    target_size=(64, 64),       # size of the images pixels X pixels
    batch_size=32,              # 32 images per batch
    class_mode='binary')        # type of outcome binary or catagorical, since we have 2 options we use binary

Found 8000 images belonging to 2 classes.


## Pre processing test set

the test data must be rescaled as well in order to increase our prediction accuracy 

In [95]:
# Creates an image data generator that will rescale our test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


# Building the CNN

## Initilising the CNN

In [96]:
cnn = tf.keras.models.Sequential()

## Step 1 Convolution

In [97]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

## Step 2 Pooling

In [98]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

## add a second conv and pooling layer

In [99]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation='relu'))


In [100]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))


## step 3 flattening

In [101]:
cnn.add(tf.keras.layers.Flatten())

## step 4 full connection

In [102]:
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

In [103]:
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Training the model

## Compiling the CNN

In [104]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Training on the training set then evaluating on the test set

In [105]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 5)
# batch_size = Integer or None. Number of samples per gradient update. If unspecified, batch_size will default to 32. Do not specify the batch_size if your data is in the form of datasets, generators, or keras.utils.Sequence instances (since they generate batches).

Epoch 1/5
250/250 [==============================] - 31s 121ms/step - loss: 0.6586 - accuracy: 0.6014 - val_loss: 0.5944 - val_accuracy: 0.6895
Epoch 2/5
250/250 [==============================] - 26s 104ms/step - loss: 0.5877 - accuracy: 0.6861 - val_loss: 0.5528 - val_accuracy: 0.7320
Epoch 3/5
250/250 [==============================] - 25s 101ms/step - loss: 0.5385 - accuracy: 0.7234 - val_loss: 0.5714 - val_accuracy: 0.7185
Epoch 4/5
250/250 [==============================] - 28s 112ms/step - loss: 0.5193 - accuracy: 0.7364 - val_loss: 0.5119 - val_accuracy: 0.7455
Epoch 5/5
250/250 [==============================] - 31s 122ms/step - loss: 0.5053 - accuracy: 0.7565 - val_loss: 0.4797 - val_accuracy: 0.7675


In [106]:
cnn.save('cnn_model')
new_model = tf.keras.models.load_model('cnn_model')

INFO:tensorflow:Assets written to: cnn_model\assets


# Making a prediction

In [108]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_4.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
print(result[0][0])
prediction = ''
if result[0][0] == 1.0:
  prediction = 'dog'
else:
  prediction = 'cat'
print(prediction)


0.0
cat


In [67]:
print(np.shape(test_image))
#(batch, images, vertical pixels, horizontal pixels)

(1, 64, 64, 3)


In [ ]:
[ (batches)
    [ (image 1)
        [ (vertical pixels)
            [ (horizontal pixels)
                [255,255,255],
                [255,255,255],
                [255,255,255],
                ...image width = 64
            ]
        ], 
        [ 
            [
                [255,255,255],
                [255,255,255],
                [255,255,255],
                ...image width = 64
            ]
        ],
        ...image height = 64 
    ],
    [ (image 2)
        [ (vertical pixels)
            [ (horizontal pixels)
                [255,255,255],
                [255,255,255],
                [255,255,255],
                ...image width = 64
            ]
        ], 
        [ 
            [
                [255,255,255],
                [255,255,255],
                [255,255,255],
                ...image width = 64
            ]
        ],
        ...image height = 64 
    ],
    ... number of images per batch = 32     
]